# Extract aperiodic component of Sleep Data 

This script loads the pre-processed polysomnography data and extracts the apeiodic component of the EEG signal using the fooof algorithm (https://github.com/fooof-tools/fooof) and the IRASA algorithm (https://doi.org/10.1007/s10548-015-0448-0). The aperiodic component is then saved as a .csv file along with other spectrally derived information and specifically utilized parameters. 

In [ ]:
%matplotlib inline

## Import packages 
import yasa
import numpy as np
import os
import mne
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
mne.set_log_level('WARNING')

In [ ]:
## 1. Load data
path = '/mnt/server/data03/2023_NENA_Aperiodic_Workshop/data/processed/'
fig_path = '/mnt/server/data03/2023_NENA_Aperiodic_Workshop/figures/subject/'
# Obtain list of unique recordings
files = list(set(["-".join(f.split('-')[0:3]) for f in os.listdir(path)]))

In [ ]:
## 2. Process and extract aperiodic component from recording files
# Iterate over all files and process them
for idx, file in enumerate(tqdm(files)):
    print(f'Detecting and labeling artifacts in file : {file}')
    if idx == 0:
        break 
    # Load the data and hypnogram files
    raw = mne.io.read_raw(path + file + '-raw.fif.gz', preload=True) # type: ignore
    hypnogram = np.load(path + file + '-hypnogram_with_art.npy')
    # Get sampling frequency
    sf = raw.info['sfreq']
    # Get data
    data = raw.get_data(['C3','C4']) * 1e6
    # Extract aperiodic component with IRASA from yasa per sleep stage
    # Get the indices of the different sleep stages

In [ ]:
# Get the indices of the different sleep stages
for stage in hypnogram:
    stage_data = data[:, hypnogram==stage]
    freqs, psd_aperiodic, psd_oscillatory, fit_params = yasa.irasa(stage_data, sf=sf, ch_names=['C3','C4'], band=(1, 30), # type: ignore
                                                                   hset=[1.1, 1.15, 1.2, 1.25, 1.3], return_fit=True, win_sec=4,
                                                                   kwargs_welch=dict(average='median', window='hamming'))

    fit_params['Stage'] = stage

In [ ]:
freqs, psd_aperiodic, psd_oscillatory, fit_params = yasa.irasa(stage_data, sf=sf, ch_names=['C3','C4'], band=(1, 30), # type: ignore
                                                               hset=[1.1, 1.15, 1.2, 1.25, 1.3], return_fit=True, win_sec=4,
                                                               kwargs_welch=dict(average='median', window='hamming'))

In [ ]:
fit_params['Stage'] = stage
fit_params

In [ ]:
# Plot the aperiodic component on a linear-log scale
plt.plot(freqs, psd_aperiodic[1, :], 'k', lw=2)
plt.xlim(1, 30)
plt.yscale('log')
sns.despine()
plt.title('Aperiodic component, chan = ' + 'C4')
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD log($uV^2$/Hz)');

In [ ]:
# Plot the oscillatory component on a linear-linear scale
plt.plot(freqs, psd_oscillatory[1, :], 'k', lw=2)
plt.xlim(1, 30)
sns.despine()
plt.title('Oscillatory component, chan = ' + 'C4')
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD log($uV^2$/Hz)');

In [ ]:
# Plot the oscillatory + aperiodic component on a linear-log scale
psd_combined = psd_aperiodic[1, :] + psd_oscillatory[1, :]
plt.plot(freqs, psd_combined, 'k', lw=2)
plt.fill_between(freqs, psd_combined, cmap='Spectral')
plt.xlim(1, 30)
plt.yscale('log')
sns.despine()
plt.title('Aperiodic + oscillatory component, chan = ' + 'C4')
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD log($uV^2$/Hz)');